# Quantum Angular Momentum Lab
## Spin, Orbital Angular Momentum, and Coupling Visualized

| Module | Topic | Key Concepts |
|--------|-------|-------------|
| **A** | Orbital Angular Momentum | $L_x, L_y, L_z$ matrices, commutators |
| **B** | Spin-1/2 | Pauli matrices, Bloch sphere |
| **C** | Addition of Angular Momenta | Clebsch-Gordan coefficients |
| **D** | Spin-Orbit Coupling | Fine structure, level splitting |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import expm
from scipy.special import sph_harm
from IPython.display import HTML

plt.style.use('seaborn-v0_8-whitegrid')
HBAR = 1.0

---
# Module A: Orbital Angular Momentum

## Angular Momentum Algebra

$$[L_x, L_y] = i\hbar L_z, \quad [L_y, L_z] = i\hbar L_x, \quad [L_z, L_x] = i\hbar L_y$$

### Ladder Operators
$$L_\pm = L_x \pm iL_y$$
$$L_\pm|l,m\rangle = \hbar\sqrt{l(l+1)-m(m\pm 1)}|l,m\pm 1\rangle$$

In [ ]:
def build_angular_momentum_operators(l, hbar=HBAR):
    """
    Build Lx, Ly, Lz matrices for angular momentum quantum number l.
    Basis: |l,m⟩ for m = -l, -l+1, ..., l-1, l (dimension 2l+1)
    """
    dim = 2*l + 1
    m_values = np.arange(-l, l+1)  # m from -l to +l
    
    # Lz is diagonal with eigenvalues m*hbar
    Lz = hbar * np.diag(m_values)
    
    # L+ (raising): L+|l,m⟩ = ℏ√[l(l+1)-m(m+1)]|l,m+1⟩
    L_plus = np.zeros((dim, dim), dtype=complex)
    for i, m in enumerate(m_values[:-1]):  # m from -l to l-1
        coeff = hbar * np.sqrt(l*(l+1) - m*(m+1))
        L_plus[i+1, i] = coeff  # |m+1⟩⟨m|
    
    # L- (lowering): L-|l,m⟩ = ℏ√[l(l+1)-m(m-1)]|l,m-1⟩
    L_minus = np.zeros((dim, dim), dtype=complex)
    for i, m in enumerate(m_values[1:], 1):  # m from -l+1 to l
        coeff = hbar * np.sqrt(l*(l+1) - m*(m-1))
        L_minus[i-1, i] = coeff  # |m-1⟩⟨m|
    
    # Lx = (L+ + L-)/2, Ly = (L+ - L-)/(2i)
    Lx = (L_plus + L_minus) / 2
    Ly = (L_plus - L_minus) / (2j)
    
    return Lx, Ly, Lz, m_values

def commutator(A, B):
    return A @ B - B @ A

In [ ]:
# Build for l=1
l = 1
Lx, Ly, Lz, m_vals = build_angular_momentum_operators(l)

print(f"Angular momentum l={l}, dimension={2*l+1}")
print(f"\nLz (diagonal with m=-1,0,+1):")
print(np.real(Lz))

# Verify commutation relations
comm_xy = commutator(Lx, Ly)
expected = 1j * HBAR * Lz
print(f"\n[Lx, Ly] = iℏLz? {np.allclose(comm_xy, expected)}")

In [ ]:
# Heatmaps of angular momentum matrices
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, (op, name) in zip(axes, [(Lx, 'Lx'), (Ly, 'Ly'), (Lz, 'Lz')]):
    # Plot real part for Lx, Lz; imaginary for Ly
    data = np.imag(op) if name == 'Ly' else np.real(op)
    im = ax.imshow(data, cmap='RdBu_r')
    ax.set_title(f'{name} ({"Im" if name=="Ly" else "Re"})')
    ax.set_xticks(range(3)); ax.set_xticklabels(['m=-1', 'm=0', 'm=+1'])
    ax.set_yticks(range(3)); ax.set_yticklabels(['m=-1', 'm=0', 'm=+1'])
    plt.colorbar(im, ax=ax)

plt.suptitle('Angular Momentum Operators (l=1)', fontsize=14)
plt.tight_layout()
plt.savefig('angular_momentum_heatmaps.png', dpi=150)
plt.show()

In [ ]:
# Spherical harmonics |Ylm(θ)|² vs θ
theta = np.linspace(0, np.pi, 100)
phi = 0  # Fix φ for 1D plot

fig, ax = plt.subplots(figsize=(10, 5))
for l_val in [0, 1, 2]:
    for m_val in range(-l_val, l_val+1):
        Y = sph_harm(m_val, l_val, phi, theta)
        prob = np.abs(Y)**2
        ax.plot(np.degrees(theta), prob, label=f'l={l_val}, m={m_val}')

ax.set_xlabel('θ (degrees)')
ax.set_ylabel('|Yₗₘ(θ,0)|²')
ax.set_title('Spherical Harmonics Probability Density')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('spherical_harmonics.png', dpi=150)
plt.show()

---
# Module B: Spin-1/2 and Pauli Matrices

## Pauli Matrices
$$\sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad
\sigma_y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad
\sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$$

## Spin Operators
$$\vec{S} = \frac{\hbar}{2}\vec{\sigma}$$

## General Spin State (Bloch Sphere)
$$|\chi(\theta,\phi)\rangle = \cos(\theta/2)|\uparrow\rangle + e^{i\phi}\sin(\theta/2)|\downarrow\rangle$$

In [ ]:
# Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

# Spin operators S = (ℏ/2)σ
Sx = (HBAR/2) * sigma_x
Sy = (HBAR/2) * sigma_y
Sz = (HBAR/2) * sigma_z

def spin_state(theta, phi):
    """General spin-1/2 state on Bloch sphere."""
    return np.array([np.cos(theta/2), np.exp(1j*phi)*np.sin(theta/2)], dtype=complex)

def bloch_vector(psi):
    """Get Bloch vector (x,y,z) from spin state."""
    x = np.real(np.vdot(psi, sigma_x @ psi))
    y = np.real(np.vdot(psi, sigma_y @ psi))
    z = np.real(np.vdot(psi, sigma_z @ psi))
    return x, y, z

In [ ]:
# Bloch sphere with sample states
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Draw sphere wireframe
u = np.linspace(0, 2*np.pi, 30)
v = np.linspace(0, np.pi, 20)
xs = np.outer(np.cos(u), np.sin(v))
ys = np.outer(np.sin(u), np.sin(v))
zs = np.outer(np.ones(len(u)), np.cos(v))
ax.plot_wireframe(xs, ys, zs, alpha=0.1, color='gray')

# Plot special states
states = {
    '|↑⟩': (0, 0), '|↓⟩': (np.pi, 0),
    '|+⟩': (np.pi/2, 0), '|−⟩': (np.pi/2, np.pi),
    '|R⟩': (np.pi/2, np.pi/2), '|L⟩': (np.pi/2, -np.pi/2)
}
colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown']

for (name, (th, ph)), color in zip(states.items(), colors):
    psi = spin_state(th, ph)
    bx, by, bz = bloch_vector(psi)
    ax.scatter([bx], [by], [bz], s=100, c=color, label=name)
    ax.quiver(0, 0, 0, bx, by, bz, color=color, arrow_length_ratio=0.1)

ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
ax.set_title('Bloch Sphere with Special States')
ax.legend(loc='upper left')
plt.tight_layout()
plt.savefig('bloch_sphere_states.png', dpi=150)
plt.show()

In [ ]:
# ANIMATION: Spin precession under H = -γB·S (B along z)
gamma = 1.0; B = 1.0
H_spin = -gamma * B * Sz  # Precession around z

psi0 = spin_state(np.pi/2, 0)  # Start on equator
times = np.linspace(0, 4*np.pi, 100)

trajectory = [bloch_vector(expm(-1j*H_spin*t/HBAR) @ psi0) for t in times]
traj = np.array(trajectory)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

def animate(frame):
    ax.clear()
    ax.plot_wireframe(xs, ys, zs, alpha=0.1, color='gray')
    ax.plot(traj[:frame+1,0], traj[:frame+1,1], traj[:frame+1,2], 'b-', lw=1.5)
    ax.quiver(0, 0, 0, traj[frame,0], traj[frame,1], traj[frame,2],
              color='red', arrow_length_ratio=0.15, linewidth=3)
    ax.set_xlim([-1.2,1.2]); ax.set_ylim([-1.2,1.2]); ax.set_zlim([-1.2,1.2])
    ax.set_title(f'Spin Precession (t = {times[frame]:.2f})')
    return []

anim = FuncAnimation(fig, animate, frames=len(times), interval=50, blit=False)
plt.close()
HTML(anim.to_jshtml())

---
# Module C: Addition of Angular Momenta

## Two Spin-1/2 Particles
Product basis: $|\uparrow\uparrow\rangle, |\uparrow\downarrow\rangle, |\downarrow\uparrow\rangle, |\downarrow\downarrow\rangle$

## Total Spin
$$\vec{S} = \vec{S}_1 + \vec{S}_2$$

### Coupled Basis
- **Triplet** ($S=1$): $|1,1\rangle, |1,0\rangle, |1,-1\rangle$
- **Singlet** ($S=0$): $|0,0\rangle$

In [ ]:
# Build total spin operators using Kronecker products
I2 = np.eye(2, dtype=complex)

# S₁ acts on first particle, S₂ on second
S1x = np.kron(Sx, I2); S1y = np.kron(Sy, I2); S1z = np.kron(Sz, I2)
S2x = np.kron(I2, Sx); S2y = np.kron(I2, Sy); S2z = np.kron(I2, Sz)

# Total spin
Stot_x = S1x + S2x
Stot_y = S1y + S2y
Stot_z = S1z + S2z

# S² = Sx² + Sy² + Sz²
S2_total = Stot_x @ Stot_x + Stot_y @ Stot_y + Stot_z @ Stot_z

print("S² eigenvalues (should be 0 and 2 for s=0 and s=1):")
eigenvalues = np.linalg.eigvalsh(S2_total)
print(np.round(eigenvalues, 3))  # s(s+1) = 0, 2 → s = 0, 1

In [ ]:
# Diagonalize S² to find singlet and triplet states
evals, evecs = np.linalg.eigh(S2_total)

product_labels = ['|↑↑⟩', '|↑↓⟩', '|↓↑⟩', '|↓↓⟩']

# Identify states by S² eigenvalue
singlet_idx = np.argmin(evals)  # s=0 → S²=0
triplet_idx = [i for i in range(4) if i != singlet_idx]

print("Singlet |0,0⟩ in product basis:")
for i, label in enumerate(product_labels):
    coeff = evecs[i, singlet_idx]
    if np.abs(coeff) > 0.01:
        print(f"  {coeff:.3f} {label}")

In [ ]:
# Bar plot of singlet/triplet decomposition
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Singlet
ax = axes[0]
singlet = evecs[:, singlet_idx]
ax.bar(range(4), np.abs(singlet)**2, color='coral')
ax.set_xticks(range(4)); ax.set_xticklabels(product_labels)
ax.set_ylabel('|Coefficient|²'); ax.set_title('Singlet |S=0, m=0⟩')
ax.set_ylim(0, 0.6)

# One triplet state (m=0)
ax = axes[1]
# Find triplet m=0 state
Sz_evals = np.diag(evecs.conj().T @ Stot_z @ evecs)
triplet_m0 = None
for i in triplet_idx:
    if np.abs(Sz_evals[i]) < 0.1:  # m=0
        triplet_m0 = evecs[:, i]
        break

if triplet_m0 is not None:
    ax.bar(range(4), np.abs(triplet_m0)**2, color='steelblue')
ax.set_xticks(range(4)); ax.set_xticklabels(product_labels)
ax.set_ylabel('|Coefficient|²'); ax.set_title('Triplet |S=1, m=0⟩')
ax.set_ylim(0, 0.6)

plt.suptitle('Clebsch-Gordan Decomposition', fontsize=14)
plt.tight_layout()
plt.savefig('clebsch_gordan.png', dpi=150)
plt.show()

---
# Module D: Spin-Orbit Coupling

## Hamiltonian
$$H = H_0 + \lambda \vec{L}\cdot\vec{S}$$

For $l=1$, $s=1/2$: 6-dimensional space.

$$\vec{L}\cdot\vec{S} = L_xS_x + L_yS_y + L_zS_z$$

In [ ]:
# Build L·S in product space |l,ml⟩ ⊗ |s,ms⟩
l = 1
Lx_orb, Ly_orb, Lz_orb, _ = build_angular_momentum_operators(l)

# Spin-1/2 operators (2×2)
I_orb = np.eye(3, dtype=complex)  # Identity for l=1
I_spin = np.eye(2, dtype=complex)

# Tensor products: L acts on orbital, S on spin
Lx_full = np.kron(Lx_orb, I_spin)
Ly_full = np.kron(Ly_orb, I_spin)
Lz_full = np.kron(Lz_orb, I_spin)
Sx_full = np.kron(I_orb, Sx)
Sy_full = np.kron(I_orb, Sy)
Sz_full = np.kron(I_orb, Sz)

# L·S operator
L_dot_S = Lx_full @ Sx_full + Ly_full @ Sy_full + Lz_full @ Sz_full

# H₀ (unperturbed): just Lz for simplicity
H0 = Lz_full

print(f"Hilbert space dimension: {L_dot_S.shape[0]} (3 orbital × 2 spin)")

In [ ]:
# Energy levels vs coupling strength λ
lambdas = np.linspace(0, 2, 100)
all_energies = []

for lam in lambdas:
    H = H0 + lam * L_dot_S
    evals = np.linalg.eigvalsh(H)
    all_energies.append(evals)

all_energies = np.array(all_energies)

fig, ax = plt.subplots(figsize=(10, 6))
for i in range(6):
    ax.plot(lambdas, all_energies[:, i], linewidth=2)

ax.set_xlabel('Spin-Orbit Coupling λ')
ax.set_ylabel('Energy')
ax.set_title('Fine Structure: Energy Level Splitting')
ax.grid(True, alpha=0.3)
ax.axvline(0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('fine_structure.png', dpi=150)
plt.show()

In [ ]:
# ANIMATION: Level splitting as λ increases
fig, ax = plt.subplots(figsize=(10, 6))

def animate(frame):
    ax.clear()
    lam_current = lambdas[frame]
    
    # Plot all curves up to current λ
    for i in range(6):
        ax.plot(lambdas[:frame+1], all_energies[:frame+1, i], linewidth=2)
    
    # Current energy levels as horizontal lines
    current_E = all_energies[frame]
    for E in current_E:
        ax.axhline(E, xmin=0.9, xmax=1.0, color='red', linewidth=3)
    
    ax.set_xlim(0, 2); ax.set_ylim(-3, 3)
    ax.set_xlabel('λ'); ax.set_ylabel('Energy')
    ax.set_title(f'Spin-Orbit Coupling (λ = {lam_current:.2f})')
    ax.grid(True, alpha=0.3)
    return []

anim = FuncAnimation(fig, animate, frames=len(lambdas), interval=50, blit=False)
plt.close()
HTML(anim.to_jshtml())

---
# Summary

| Module | Key Results |
|--------|-------------|
| **A** | $[L_x, L_y] = i\hbar L_z$ verified numerically |
| **B** | Bloch sphere maps spin states; precession visualized |
| **C** | Singlet/triplet from CG decomposition |
| **D** | Spin-orbit splits degenerate levels |